# Градиентный спуск (15 баллов)

В этой домашней работе мы попробуем разобраться в том, что же такое градиентный спуск, как он работает и что можно делать с его помощью. Обычно для этих целей используются пара готовых классов из `sklearn`: `SGDClassifier` и `SGDRegressor`, на основании которых написано еще много чего интересного. Можно было бы воспользоваться этими классами и, подражая обезьяне, пробовать крутить различные ручки-параметры, пытаясь понять, что же они значат и как работают. Но это не наш путь, поэтому мы напишем все сами.

## Что будем делать

Градиентный спуск это основной метод оптимизации в машинном обучении, и он нам еще не раз пригодится. Чтобы не терять свои наработки в дюжине тетрадок, мы напишем свою упрощенную версию библиотеки `sklearn`, добавляя туда функционал от домашки к домашке. Скелет нашего фреймворка в самом начале будет крутиться вокруг трех классов:

`_losses.py` -- функции потерь для различных линейных моделей. Каждая функция потерь имеет два метода: `loss`, который непосредственно вычисляет значение функции потерь, и `dloss`, который вычисляет значение ее производной (градиента). В качестве аргументов эти функции принимают предсказанное значение `p` и истинное значение `y` для объекта.

`_sgd.py` -- основа основ, тут находится метод `sgd`, который для полученной лосс функции проходит `epochs` итераций градиентного спуска, обновляя полученные веса и смещения на основании входных данных. Основную часть работы мы будем вести здесь.

`SGDRegressor.py` -- обертка над `sgd`, обрабатывает входные данные и реализует интерфейсы, принятые в `sklearn`.

# SquaredLoss (0.5 балла)

Чтобы в домашку было проще вкатится, мы начнем с малого, а именно -- реализуем квадратичную функцию потерь. Как мы помним из теории, квадратичная функция потерь на вход принимает два числа (`p` и `y` в нашем случае), а зачем вычисляется по следующей формулe:

$SE = (y - \hat{y})^2$, где $\hat{y}$ это наше предсказание, или $p$.

Для каждого $i$-го объекта лосс вычисляется по следующей формуле:

$L\left(y_i, f(x_i)\right) = \frac{1}{2}\left(y_i - f(x_i)\right)^2$ или $L(y_i, p_i) = \frac{1}{2}\left(y_i - p_i\right)^2$

Для начала реализуйте эту формулу и ее производную в заранее подготовленном классе `SquaredLoss`, который можно найти в `_losses.py`. Места где от вас хотят что-то увидеть помечены строчкой `<YOUR CODE HERE>`. Сам класс выглядит примерно вот так:

Ячейку ниже править не надо, она только для примера, пишите код в `_losses.py`.

In [ ]:
class SquaredLoss(RegressionLoss):
    def loss(self, p: float, y: float) -> float:
        # <YOUR CODE HERE>

    def dloss(self, p: float, y: float) -> float:
        # <YOUR CODE HERE>

После этого, чтобы удостоверится в правильности реализации формул, прогоните тесты для проверки. Это можно сделать как в ячейке ниже, так и в консоли, запустив следующую команду:

`pytest test_sgd.py -k TestSquaredLoss -v`

Для запуска в ячейке перед командой введите восклицательный знак (просто запустите ячейку ниже, он там уже есть). Для проверки конечно было бы удобно, если бы все такие ячейки были выполнены.

In [8]:
!pytest test_sgd.py -k TestSquaredLoss -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 71 deselected / 2 selected                                

test_sgd.py::TestSquaredLoss::test_loss PASSED                           [ 50%]
test_sgd.py::TestSquaredLoss::test_dloss PASSED                          [100%]

======================= 2 passed, 71 deselected in 0.57s =======================


Как понять, что все прошло успешно? Есть несколько признаков

* `test_gd.py ..` -- справа от имени файла с тестом только точки, никаких букв `F`;
* `[100%]` -- еще правее цифра 100%;   
* В полосе снизу написано `2 passed`, нет слова `failed` и она зеленого цвета, а не красного. 

Если что-то пошло не так, проверьте свой код и посмотрите в тесты еще раз. Повторяйте до сходимости, после чего переходите к следующему пункту.

# SGD (2 балла)

Теперь мы перейдем к основной части нашей работы -- алгоритму стохастического градиентного спуска. Заготовка функции, в которой мы его реализуем лежит в `sgd.py`. Не пугайтесь, хоть там и много аргументов, все они описаны в докблок комментарии в начале функции. Нас сейчас будут интересовать далеко не все, а только самые необходимые для работы:

* `weights` -- вектор весов линейной модели (если кто помнит, их количество равно количеству признаков);
* `intercept` -- bias, смещение плоскости относительно нуля;
* `loss` -- класс, реализующий интерфейс функции потерь, такую мы уже реализовали выше;
* `X` -- список объектов из тренировочной выборки;
* `y` -- список таргетов для этих объектов;
* `max_iter` -- максимальное количество итераций (или шагов, или *эпох*) градиентного спуска;
* `fit_intercept` -- обучать ли смещение (intercept) или нет
* `eta0` -- под этим странным именем скрывается скорость обучения, или *learning rate*.

Для решения этой задачи напомню алгоритм стохастического градиентного спуска в рамках регрессии, которую мы решаем:

## Математическая формулировка

Дано множество объектов $(x_1, y_1), ..., (x_n, y_n)$, где $x_i\in\mathbb{R}^m$ и $y_i\in\mathbb{R}$. Наша цель -- обучить линейную модель $f(x) = w^Tx + b$ с весами $w\in\mathbb{R}^m$ и смещением $b\in\mathbb{R}$. Для того, чтобы найти эти параметры, мы минимизируем регуляризованную ошибку на тренировочной выборке:

$E(w, b) = \frac{1}{n}\sum_{i=1}^{n}L(y_i, f(x_i)) + \alpha R(w)$,

где $L$ это функция потерь, а $R$ это регуляризатор. $\alpha > 0$ это неотрицательный параметр, который контролирует силу регуляризации. В этой домашней работе мы будем считать, что $\alpha = 0$ и регуляризация не используется.

В качестве алгоритма минимизации используется стохастический градиентный спуск (stochastic gradient descent, SGD). SGD аппроксимирует истинное значение ошибки $E(w, b)$ рассматривая по одному объекту из тренировочный выборки за раз. Алгоритм перебирает все объекты тренировочный выборки и для каждого обновляет параметры модели в соответствии с правилами, описанными следующими формулами:

$w\leftarrow w -\eta\left[\frac{\partial{L(w^Tx_i + b, y_i)}}{\partial{w}}\right]$

$b\leftarrow b -\eta\left[\frac{\partial{L(w^Tx_i + b, y_i)}}{\partial{b}}\right]$

На более понятном языке это можно выразить так:
1. Вычислите значение линейной функции для $x_i$ объекта;
2. Вычислите градиент функции потерь (часть этого уже готова в предыдущем пункте);
3. Обновите параметры модели $w$ и $b$ (`weights` и `intercept`). Не забудьте про learning rate.

После того, как все покажется максимально понятным, реализуйте это в коде. Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest test_sgd.py -k TestSgdFn -v`

In [9]:
!pytest test_sgd.py -k TestSgdFn -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 69 deselected / 4 selected                                

test_sgd.py::TestSgdFn::test_return_epochs_with_no_early_stopping PASSED [ 25%]
test_sgd.py::TestSgdFn::test_fit_only_weights PASSED                     [ 50%]
test_sgd.py::TestSgdFn::test_fit_weights_fit_intercept PASSED            [ 75%]
test_sgd.py::TestSgdFn::test_fit_given_weights PASSED                    [100%]

=============================== warnings summary ===============================
test_sgd.py::TestSgdFn::test_return_epochs_with_no_early_stopping
test_sgd.py::TestSgdFn::test_fit_only_weights
test_sgd.py::TestSgdFn::test_fit_weights_fit_intercept
test_sgd.py::TestSgdFn::test_fit_given_weights
  /home/maksim/

# SGDRegressor (2 балла)

После того, как мы написали ядро модели, напишем и обертку, подражая интерфейсу `sklearn.linear_model.SGDRegressor`. Для начала стоит определить конструктор класса, который принимает параметры `loss`, `fit_intercept`, `max_iter` и `eta0`. Обратите внимание, что для удобства лосс функцию нужно передавать строкой, а на основании этого решать, какой класс для ее вычисления использовать.

Обратите внимание, что все таким классы имеют по умолчанию значения для ***всех*** параметров. Это нужно, чтобы с таким интерфейсом было удобно работать из коробки, в несколько секунд создавая модель:

`reg = SGDRegressor()`

Это удобно, ведь мы можем каждый раз переопределять только те параметры, которые нам нужны. По для значения `fit_intercept` по умолчанию нужно выставить `True`, для `max_iter` -- тысячу, а для `eta0` -- одну сотую.

Интерфейс практически любой модели машинного обучения с учителем в `scikit-learn` имеет два главных метода:  
`fit(X, y)` -- принимает матрицу объекты-признаки и вектор таргетов тренировочной выборки, запускает процесс обучения;  
`predict(X)` -- принимает матрицy объекты-признаки тестовой выборки, выдает предсказания для полученных объектов. 

В этой части нужно будет реализовать методы конструктор класса, а также методы `fit` и `predict`. Задача тут стоит простая -- инициализировать параметры модели и передать их в уже готовый написанный метод `sgd`. Цель всего этого -- просто понять, как разделяется ответственность между функцией с алгоритмом и обвязкой в виде класса.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest test_sgd.py -k TestSGDRegressor -v`

In [10]:
!pytest test_sgd.py -k TestSGDRegressor -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 66 deselected / 7 selected                                

test_sgd.py::TestSGDRegressor::test_validate_loss PASSED                 [ 14%]
test_sgd.py::TestSGDRegressor::test_validate_max_iter PASSED             [ 28%]
test_sgd.py::TestSGDRegressor::test_coef_attribute PASSED                [ 42%]
test_sgd.py::TestSGDRegressor::test_intercept_attribute PASSED           [ 57%]
test_sgd.py::TestSGDRegressor::test_fit_only_weights PASSED              [ 71%]
test_sgd.py::TestSGDRegressor::test_fit_weights_fit_intercept PASSED     [ 85%]
test_sgd.py::TestSGDRegressor::test_predict SKIPPED (not implemented)    [100%]

=============================== warnings summary =====================

Если все тесты проходят, то вы справились с этой задачей. Поздравляю, вы написали свою первую модель машинного обучения руками с нуля, а это все-таки нетривиальная задачка.

# Взрыв градиента (gradient explosion) (0.5 балла)

Давайте испробуем нашу самописную модель на реальных данных и поглядим, как оптимизируются веса модели.

In [11]:
from sklearn.datasets import load_boston
from SGDRegressor import SGDRegressor

X, y = load_boston(return_X_y=True)
reg = SGDRegressor().fit(X, y)
reg.coef_

/home/maksim/PycharmProjects/twinkle/03-sgd/_sgd.py:43: RuntimeWarning: overflow encountered in multiply
  dp_dw = (y_hat - y[i])*X[i]*sample_weight[i] if sample_weight is not None else (y_hat - y[i])*X[i]
/home/maksim/PycharmProjects/twinkle/03-sgd/_sgd.py:43: RuntimeWarning: invalid value encountered in multiply
  dp_dw = (y_hat - y[i])*X[i]*sample_weight[i] if sample_weight is not None else (y_hat - y[i])*X[i]
/home/maksim/PycharmProjects/twinkle/03-sgd/_sgd.py:45: RuntimeWarning: invalid value encountered in subtract
  weights = weights - eta*dp_dw


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

Огонь! Не переживайте, если вы увидите здесь вектор `nan` (а вы его здесь увидите). Если тесты прошли -- вы все сделали правильно,поэтому давайте подумаем, как мы пришли к жизни такой. Для этого посмотрим, как изменяются веса в зависимости от количества пройденных эпох.

*Кстати, если вам повезет, то вы увидите красное предупреждение о недопустимом значении при умножении в вычислении весов.*

Для удобства отладки добавим еще один булев параметр в класс `SGDRegressor`, а называется он `verbose`. Он будет отвечать за *многословность* нашей модели, и в verbose-режиме модель будет выдавать отладочную информацию. Этот же параметр прокиньте в функцию `sgd`, благо там он уже есть, и даже есть пример его использования.

После этого добавьте отладочный вывод градиента `dloss` на каждом ***объекте*** при включенном параметре `verbose` (не забудьте передать его в вызове `sgd` внутри метода `fit`). Для этого воспользуйтесь заготовкой функции `print_dloss` в том же файле `_sgd.py`. Чтобы не выводить пачку лишних nan, добавьте в функции условие проверку на nan наравне с `verbose`. Здесь вам предстоит потренироваться в сложнейшем написании условия и форматированном выводе питона, чтобы все выглядело красиво.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest test_sgd.py -k TestPrintDloss -v`

In [12]:
!pytest test_sgd.py -k TestPrintDloss -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 65 deselected / 8 selected                                

test_sgd.py::TestPrintDloss::test_no_verbose PASSED                      [ 12%]
test_sgd.py::TestPrintDloss::test_nan PASSED                             [ 25%]
test_sgd.py::TestPrintDloss::test_zero PASSED                            [ 37%]
test_sgd.py::TestPrintDloss::test_positive1 PASSED                       [ 50%]
test_sgd.py::TestPrintDloss::test_positive2 PASSED                       [ 62%]
test_sgd.py::TestPrintDloss::test_positive3 PASSED                       [ 75%]
test_sgd.py::TestPrintDloss::test_negative PASSED                        [ 87%]
test_sgd.py::TestPrintDloss::test_inside_sgd_no_verbose SKIPPED (not...

Ну а теперь еще разок запустим обучение на одну эпоху и включенным параметром `verbose`:

*Подсказка: если выводятся только номера эпох, а градиентов нет -- перезапустите тетрадку: сверху в меню Kernel: Resart.*

In [13]:
from sklearn.datasets import load_boston
from SGDRegressor import SGDRegressor

X, y = load_boston(return_X_y=True)
reg = SGDRegressor(max_iter=1, verbose=True).fit(X, y)

-- Epoch 1
-- grad -7.60e+02
-- grad +1.70e+06
-- grad -3.71e+09
-- grad +6.27e+12
-- grad -9.84e+15
-- grad +2.88e+19
-- grad -1.02e+23
-- grad +2.18e+26
-- grad -6.77e+29
-- grad +2.48e+33
-- grad -6.68e+36
-- grad +1.65e+40
-- grad -3.85e+43
-- grad +8.42e+46
-- grad -1.88e+50
-- grad +7.11e+53
-- grad -2.55e+57
-- grad +6.34e+60
-- grad -1.41e+64
-- grad +3.08e+67
-- grad -7.51e+70
-- grad +2.02e+74
-- grad -4.98e+77
-- grad +1.27e+81
-- grad -3.45e+84
-- grad +6.41e+87
-- grad -1.35e+91
-- grad +6.49e+94
-- grad -2.69e+98
-- grad +6.71e+101
-- grad -1.68e+105
-- grad +4.33e+108
-- grad -1.55e+112
-- grad +5.37e+115
-- grad -1.86e+119
-- grad +8.58e+122
-- grad -2.68e+126
-- grad +6.82e+129
-- grad -1.68e+133
-- grad +4.21e+136
-- grad -1.02e+140
-- grad +2.38e+143
-- grad -6.02e+146
-- grad +1.37e+150
-- grad -2.10e+153
-- grad +9.83e+156
-- grad -4.26e+160
-- grad +1.45e+164
-- grad -3.40e+167
-- grad +7.48e+170
-- grad -1.65e+174
-- grad +4.82e+177
-- grad -1.45e+181
-- grad +3.

А теперь посмотрите на значения градиента и напишите ниже ответы на несколько вопросов:
* Как изменяется значение градиента?
* Какое последнее значение лосса выведено, до того как стало `nan`?
* Что просходит с весами при таких значениях градиента?
* Почему происходит взрыв градиента?
* Как бороться с этим взрывом?

* Меняет знак
* -4.95e+306
* Уходят в бесконечность
* Признаки разные порядков
* Масштабирование

# Масштабирование признаков (5 баллов)

## MaxAbsScaler (1 балл)

Как мы уже поняли, недостатком нашего алгоритма является чувствительность к масштабированию признаков. Немасштабированные признаки могут иметь огромные значения от минус до плюс бесконечности. Самый простой способ уменьшить их разброс -- найти максимальное значение каждого признака среди всех объектов, запомнить его, после чего разделить значение каждого признака на это максимальное значение. После этого значения признаков будет в диапазоне $[-1.0, 1.0]$, а максимальное абсолютное значение каждого признака не будет превышать $1.0$.

Реализуйте эту логику в классе `MaxAbsScaler`, заготовку которого можно найти в файле `MaxAbsScaler.py`. Что делают тесты:
1. `test_fit_chainable` проверяет, что метод `fit` возвращает указатель на сам объект модели;
2. `test_fit_n_samples_seen` проверяет, что вызов `fit` записывает в атрибут `n_samples_seen_` объекта модели количество объектов в обучающей выборке;
3. `test_fit_max_abs_*` проверяют, что метод `fit` записывает в атрибут `max_abs_` вектор максимальных по модулю значений каждого признака;
4. `test_fit_scale_*` проверяют, что метод `fit` записывает в атрибут `scale_` вектор масштабов для приведения признаков в диапазон $[-1.0, 1.0]$. В дальнейшем этот вектор поэлементно домножается на вектор признаков, который мы хотим отмасштабировать;
5. `test_transform_*` проверяют, что метод `transform`, верно масштабирует матрицу объектов на входе с помощью обученных параметров скейлера;
5. `test_fit_transform_*` проверяют, что метод `fit_transform` работает одновременно и как `fit` (обучает параметры модели), и как `transform` (преобразует входные данные).

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestMaxAbsScaler -v`

In [14]:
!pytest test_sgd.py -k TestMaxAbsScaler -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 63 deselected / 10 selected                               

test_sgd.py::TestMaxAbsScaler::test_fit_chainable PASSED                 [ 10%]
test_sgd.py::TestMaxAbsScaler::test_fit_n_samples_seen PASSED            [ 20%]
test_sgd.py::TestMaxAbsScaler::test_fit_max_abs_easy PASSED              [ 30%]
test_sgd.py::TestMaxAbsScaler::test_fit_max_abs_from_docs PASSED         [ 40%]
test_sgd.py::TestMaxAbsScaler::test_fit_scale_easy PASSED                [ 50%]
test_sgd.py::TestMaxAbsScaler::test_fit_scale_from_docs PASSED           [ 60%]
test_sgd.py::TestMaxAbsScaler::test_transform_easy PASSED                [ 70%]
test_sgd.py::TestMaxAbsScaler::test_transform_from_docs PASSED         

Давайте попробуем воспользоваться нашим скейлером, чтобы решить задачу регрессии на тех же данных, что и раньше.

In [15]:
from sklearn.datasets import load_boston
from MaxAbsScaler import MaxAbsScaler
from SGDRegressor import SGDRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

X, y = load_boston(return_X_y=True)
X = MaxAbsScaler().fit_transform(X)
reg = SGDRegressor(max_iter=1).fit(X, y)
reg.coef_

array([-0.275884  ,  4.50347168, -1.49808335,  2.68848854,  2.25574396,
        7.17742612,  0.4795809 ,  4.18624003, -2.44051075, -0.53361557,
        3.45940077,  8.06658573, -2.91344649])

Работает ли теперь наша модель и взрываются ли веса? Почему? Напишите, что вы думаете об этом:

Работает, веса не взрываются, поскольку признаки одинаковых порядков

## Метрики качества регрессии: MAE и MSE (1 балл)

Теперь, когда все хоть как-то работает, стоит измерить качество модели. Для этого воспользуемся всем известными средней абсолютной и квадратичной ошибками, измерять которые будем на отложенной выборке.

Реализуйте эти две метрики в файле `metrics.py` в соответствующих заготовках функций `mean_absolute_error` и `mean_squared_error`. Что делают тесты:
1. `test_absolute_easy` проверяет, что функция `mean_absolute_error` работает корректно;
1. `test_squared_easy` проверяет, что функция `mean_squared_error` работает корректно;

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestMetrics -v`

In [16]:
!pytest test_sgd.py -k TestMetrics -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 71 deselected / 2 selected                                

test_sgd.py::TestMetrics::test_absolute_easy PASSED                      [ 50%]
test_sgd.py::TestMetrics::test_squared_easy PASSED                       [100%]

======================= 2 passed, 71 deselected in 0.60s =======================


Разбейте выборку на трейн и тест, обучите модель на трейне с параметрами по умолчанию, получите предсказания на тестовой выборке. Посчитайте метрики на отложенной выборке, выведите их значения.

*Подсказка: после этого заверните все это в функцию с одним параметром, через который в нее можно передать объект скейлера. Она нам еще пригодится.*

In [17]:
from sklearn.model_selection import train_test_split
from metrics import mean_squared_error, mean_absolute_error


def model_scale(X, y, scaler=False):
    if scaler:
        print("SCALED with", scaler)
        X = scaler().fit_transform(X)
  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    reg = SGDRegressor(max_iter=1).fit(X_train, y_train)
    prediction = reg.predict(X_test)
    mse = mean_squared_error(prediction, y_test)
    mae = mean_absolute_error(prediction, y_test)
    print(f"MSE = {mse}\nMAE = {mae}\n\n")
model_scale(X, y)

MSE = 57.38077724144307
MAE = 5.267689670344145




Напишите ниже ответы на следующие вопросы:
* Что вы думаете о полученных значениях?
* Много это или мало, хорошо или плохо?
* На чем лучше фитить скейлер: на всем датасете или только на трейне?

In [18]:
y.mean()

22.532806324110677

* Кажется, довольно большая ошибка
* Лучше скейлить весь датасет 

## MinMaxScaler (1 балл)

Предыдущий способ масштабирования работает, и работает относительно неплохо -- по крайней мере модель как-то обучается. Хорошо она обучается или плохо мы посмотрим чуть позже, а пока займемся написанием еще пары классов для маcштабирования признаков.

`MinMaxScaler` масштабирует признаки в заданный диапазон значений `[min, max]`, по умолчанию это `[0, 1]`, который передается как параметр конструктора -- `feature_range`. Работает скейлер весьма просто. Для начала нужно найти масштаб: взять разность между максимальным и минимальным значениями диапазона, в который мы масштабируем (`feature_range`), а потом разделить её на разность максимального и минимального значений по каждому признаку. После этого нужно отмасштабировать минимальные значения каждого признака и вычесть их из начала заданного диапазона, в который мы масштабируем. Потом для преобразования данных мы просто должны умножить каждый признак на масштаб и прибавить к нему минимальные значения из предыдущего шага.

Реализуйте эту логику в классе `MinMaxScaler`, заготовку которого можно найти в файле `MinMaxScaler.py`. Что делают тесты:
1. `test_fit_chainable` проверяет, что метод `fit` возвращает указатель на сам объект модели;
2. `test_fit_n_samples_seen` проверяет, что вызов `fit` записывает в атрибут `n_samples_seen_` объекта модели количество объектов в обучающей выборке;
3. `test_fit_data_min` проверяет, что метод `fit` записывает в атрибут `data_min_` вектор минимальных значений каждого признака;
4. `test_fit_data_max` проверяет, что метод `fit` записывает в атрибут `data_max_` вектор максимальных значений каждого признака;
5. `test_fit_data_range` проверяет, что метод `fit` записывает в атрибут `data_range_` вектор разницы между максимальными и минимальными значениями каждого признака, см. пункты выше;
6. `test_fit_scale` проверяет, что метод `fit` записывает в атрибут `scale_` масштаб для каждого признака. Он вычисляется как отношение разницы максимального и минимального значений данного диапазона `feature_range` из конструктора к разбросу значений на данных из предыдущего пункта. *Подсказка: если вам где-то захочется поделить на ноль, замените его единичкой*;
7. `test_fit_min` проверяет, что метод `fit` записывает в атрибут `min_` вектор значений для корректировки к минимуму. Он вычисляется как разность минимального значения из диапазона `feature_range` и отмасштабированных минимальных значений каждого признака;
8. `test_fit_in_feature_range` проверяет, что метод `fit` корректно работает при передаче диапазона значений, отличных от значений по умолчанию $[0, 1]$;
9. `test_transform` проверяет, что метод `transform` корректно масштабирует данные, используя обученные параметры скейлера;
10. `test_fit_transform` проверяет, что метод `fit_transform` корректно обучает параметры скейлера и корректно масштабирует полученные данные.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestMinMaxScaler -v`

In [19]:
!pytest test_sgd.py -k TestMinMaxScaler -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 63 deselected / 10 selected                               

test_sgd.py::TestMinMaxScaler::test_fit_chainable PASSED                 [ 10%]
test_sgd.py::TestMinMaxScaler::test_fit_n_samples_seen PASSED            [ 20%]
test_sgd.py::TestMinMaxScaler::test_fit_data_min PASSED                  [ 30%]
test_sgd.py::TestMinMaxScaler::test_fit_data_max PASSED                  [ 40%]
test_sgd.py::TestMinMaxScaler::test_fit_data_range PASSED                [ 50%]
test_sgd.py::TestMinMaxScaler::test_fit_scale PASSED                     [ 60%]
test_sgd.py::TestMinMaxScaler::test_fit_min PASSED                       [ 70%]
test_sgd.py::TestMinMaxScaler::test_fit_in_feature_range PASSED        

## StandardScaler (1 балл)

К третьему скейлеру вы должны были задуматься над следующим вопросом: "Зачем нам столько разных?". Мы сравним их чуть позже, а пока давайте допишем последний. Он масштабирует признаки вычитая среднее и деля на стандартное отклонение по следующей формуле:

$z = \frac{(x - u)}{s}$, где $u$ это среднее значение признака по выборке, а $s$ это стандартное отклонение этого признака.

Этот скейлер должен принимать два аргумента:  
`with_mean` -- включает вычисление среднего, иначе среднее -- 0;  
`with_std` -- включает вычисление стандартного отклонения, иначе отклонение -- 1.

Реализуйте эту логику в классе `StandardScaler`, заготовку которого можно найти в файле `StandardScaler.py`. Что делают тесты:
1. `test_fit_chainable` проверяет, что метод `fit` возвращает указатель на сам объект модели;
2. `test_fit_n_samples_seen` проверяет, что вызов `fit` записывает в атрибут `n_samples_seen_` объекта модели количество объектов в обучающей выборке;
3. `test_fit_mean` проверяет, что метод `fit` записывает в атрибут `mean_` вектор средних значений по каждому признаку;
4. `test_fit_var` проверяет, что метод `fit` записывает в атрибут `var_` вектор стандартных отклонений по каждому признаку;
5. `test_fit_var_without_std` проверяет, что метод `fit` записывает в атрибут `var_` значение `None` при выключенном флаге `with_std`;
6. `test_fit_mean_var_without_mean_std` проверяет, что метод `fit` записывает в атрибуты `mean_` и `var_` значения `None` при выключенных флагах `with_mean` и `with_std`;
7. `test_fit_scale` проверяет, что метод `fit` записывает в атрибут `scale_` вектор масштабов каждого признака;
8. `test_fit_scale_without_std` проверяет, что метод `fit` записывает в атрибут `scale_` значение `None` при выключенном флаге `with_std`;
9. `test_fit_scale_without_mean_std` проверяет, что метод `fit` записывает в атрибут `scale_` значение `None` при выключенных флагах `with_mean` и `with_std`;
10. `test_transform` проверяет, что метод `transform` корректно масштабирует данные, используя обученные параметры скейлера;
11. `test_transform_without_std` проверяет, что метод `transform` корректно масштабирует данные, используя обученные параметры скейлера (при выключенном параметре флаге `with_std`);
12. `test_transform_without_mean_std` проверяет, что метод `transform` корректно масштабирует данные, используя обученные параметры скейлера (при выключенных флагах `with_mean` и `with_std`);
13. `test_fit_transform` проверяет, что метод `fit_transform` корректно обучает параметры скейлера и корректно масштабирует полученные данные.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestStandardScaler -v`

In [20]:
!pytest test_sgd.py -k TestStandardScaler -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 60 deselected / 13 selected                               

test_sgd.py::TestStandardScaler::test_fit_chainable PASSED               [  7%]
test_sgd.py::TestStandardScaler::test_fit_n_samples_seen PASSED          [ 15%]
test_sgd.py::TestStandardScaler::test_fit_mean PASSED                    [ 23%]
test_sgd.py::TestStandardScaler::test_fit_var PASSED                     [ 30%]
test_sgd.py::TestStandardScaler::test_fit_var_without_std PASSED         [ 38%]
test_sgd.py::TestStandardScaler::test_fit_mean_var_without_mean_std PASSED [ 46%]
test_sgd.py::TestStandardScaler::test_fit_scale PASSED                   [ 53%]
test_sgd.py::TestStandardScaler::test_fit_scale_without_std PASSED   

## Чем лучше масштабировать? (1 балл)

Давайте сравним наши способы масштабировать признаки, обучив регрессор со всеми тремя алгоритмами. Вычислите MAE и MSE для каждого скейлера на тестовых данных, сравните их.

In [21]:
from StandardScaler import StandardScaler
from MinMaxScaler import MinMaxScaler
from MaxAbsScaler import MaxAbsScaler

X, y = load_boston(return_X_y=True)

def model_scale(X, y, scaler=False):
    if scaler:
        print("SCALED with", scaler)
        X_scaled = scaler().fit_transform(X)
    else:
        X_scaled = X
  
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    reg = SGDRegressor(max_iter=1).fit(X_train, y_train)
    prediction = reg.predict(X_test)
    mse = mean_squared_error(prediction, y_test)
    mae = mean_absolute_error(prediction, y_test)
    print(f"MSE = {mse}\nMAE = {mae}\n\n")
    
model_scale(X, y, scaler=MinMaxScaler)
model_scale(X, y, scaler=StandardScaler)
model_scale(X, y, scaler=MaxAbsScaler)

SCALED with <class 'MinMaxScaler.MinMaxScaler'>
MSE = 55.32783352598604
MAE = 5.419377635858724


SCALED with <class 'StandardScaler.StandardScaler'>
MSE = 27.32460800735155
MAE = 3.336798799196438


SCALED with <class 'MaxAbsScaler.MaxAbsScaler'>
MSE = 53.10457081045337
MAE = 5.170841355783048




Ответьте на следующие вопросы:
* Сильно ли отличаются результаты?
* Появился ли смысл в значениях средней абсолютной и квадратичной ошибок в рамках этой задачи?
* Какой метод масштабирования работает лучше всех?
* Какой хуже всех? Как вам кажется, почему?
* Каким будете пользоваться в дальнейшем? 

* Довольно сильно
* MSE больше штрафует за выбросы 
* StandardScaler
* MaxAbsScaler и MinMaxScaler работают аналогично
* Зависит от задачи

# Бонус: снова SGD (5 баллов)

Настало время расширить наш игрушечный алгоритм, доведя его до ума. Для этого надо добавить еще несколько фич.

## Случайный выбор объектов. Shuffle (1 балл)

Пока что наша модель не совсем соответствует алгоритму стохастического градиентного спуска, ведь мы перебираем элементы всегда в одном и том же порядке. В этой части вам необходимо добавить случайность в выборе элементов внутри одной эпохи при условии, что каждый элемент будет выбран ровно один раз.

Однако перемешивание элементов дело непростое, и для удобства тестирование должно быть воспроизводимым. Для этого нужно дополнительно передавать в модель зерно генератора псевдослучайных чисел. Добавьте в конструктор `SGDRegressor` булев параметр `shuffle` и сделайте его по умолчанию равным `True`. Еще добавьте целочисленный параметр `random_state`, равный по умолчанию `None`. После прокиньте их в функцию `sgd` внутри метода `fit`.

Внутри `sgd` в зависимости от флага `shuffle` создавайте ГПСЧ с заданным зерном и добавьте случайный выбор следующего элемента. Зерно генератора случайных чисел соответствует параметру `seed` внутри функции `_sgd`. Помните, что каждый элемент должен поучаствовать в градиентном спуске один раз в каждой эпохе.

*Подсказка: проще сначала написать реализацию в функции `sgd`, простестировать ее, и после прокинуть параметры снаружи из `SGDRegressor`.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestShuffle -v`

In [22]:
!pytest test_sgd.py -k TestShuffle -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 66 deselected / 7 selected                                

test_sgd.py::TestShuffle::test_sgd_every_item PASSED                     [ 14%]
test_sgd.py::TestShuffle::test_sgd_no_shuffle PASSED                     [ 28%]
test_sgd.py::TestShuffle::test_sgd_with_given_seed PASSED                [ 42%]
test_sgd.py::TestShuffle::test_sgd_with_empty_seed PASSED                [ 57%]
test_sgd.py::TestShuffle::test_reg_no_shuffle PASSED                     [ 71%]
test_sgd.py::TestShuffle::test_reg_with_given_seed PASSED                [ 85%]
test_sgd.py::TestShuffle::test_reg_with_empty_seed PASSED                [100%]

=============================== warnings summary =====================

## Дообучение модели. Partial fit (1 балл)

В `scikit-learn` некоторые модели включают в себя себя следующий метод:

`partial_fit(X, y)` -- принимает матрицу объекты-признаки и вектор таргетов, обновляет веса модели, проходя градиентным спуском по новой пачке данных. Внутри этот метод реализуется как тот же `fit`, только не обнуляет веса *(параметры модели)* и прогоняет ***одну итерацию*** градиентного спуска по новым данным.

В этой части реализуйте метод `partial_fit(X, y)`, который прогоняет одну эпоху градиентного спуска по своим аргументам, пользуясь уже обученными параметрами модели. Вы можете создать новый приватный метод `__partial_fit` (начинается с подчеркивания), который наравне с матрицей объекты-признаки и вектором таргетов принимает количество эпох. В него можно перетащить содержимое `fit`, а дальше уже использовать этот новый метод как внутри `fit` с (`max_iter=max_iter`), так и внутри `partial_fit` (c `max_iter=1`). Таким образом не придется делать два больших вызова `sgd` дважды.

Реализуйте эту логику в классе `SGDRegressor`. Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestPartialFit -v`

In [23]:
!pytest test_sgd.py -k TestPartialFit -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 67 deselected / 6 selected                                

test_sgd.py::TestPartialFit::test_chainable PASSED                       [ 16%]
test_sgd.py::TestPartialFit::test_coef_shape PASSED                      [ 33%]
test_sgd.py::TestPartialFit::test_intercept_shape PASSED                 [ 50%]
test_sgd.py::TestPartialFit::test_runs_one_iter_only PASSED              [ 66%]
test_sgd.py::TestPartialFit::test_use_fitted_weights FAILED              [ 83%]
test_sgd.py::TestPartialFit::test_use_fitted_intercept FAILED            [100%]

=================================== FAILURES ===================================
____________________ TestPartialFit.test_use_fitted_weights _________

## Горячий старт. Warm start (1 балл)

Иногда обучение модели занимает достаточно очень много времени. Представим ситуацию: мы подбираем гиперпараметры модели, и на каждом этапе не очень то хочется учить модель с нуля, ведь нам достаточно обучить одну модель, а все остальные начинать обучать уже не с начала, а используя параметры обученной модели. С такой же ситуацией можно столкнуться, когда мы закончили обучение, но нам хочется прогнать еще сотню-другую эпох в надежде, что качество вырастет. Сейчас в нашей модели можем вызывать метод `fit` заново, но это приводит к паре проблем:
1. Нет возможности выучить ту же модель на новых данных с нуля (со сбросом параметров к начальным);
2. Нет возможности передать веса в модель перед обучением.

Итак, хочется иметь возможность брать параметры обученной модели и передавать их в новую модель. Как же это можно сделать? Давайте добавим в конструктор булев параметр `warm_start` (по умолчанию `False`), и в зависимости от него будем решать, оставлять или сбрасывать параметры модели при перезапусках `fit`. Кроме того, нужно добавить в `fit` два параметра: `coef_init` и `intercept_init`, оба по умолчанию `None`. В них можно будет передать начальные значения весов и смещение.

Алгоритм будет прост: если параметр `warm_start` включен, то при перезапуске `fit` мы пытаемся взять аргументы `coef_init` и `intercept_init` в качестве исходных значений весов и смещений. Если один из них пуст, то берем в качестве исходных уже готовые параметры нашей модели. Если `warm_start` выключен, то генерируем параметры заново, как мы уже делали до этого.

Реализуйте эту логику в классе `SGDRegressor`. Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestWarmStart -v`

In [24]:
!pytest test_sgd.py -k TestWarmStart -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 72 deselected / 1 selected                                

test_sgd.py::TestWarmStart::test_disabled SKIPPED (not implemented)      [100%]

====================== 1 skipped, 72 deselected in 0.59s =======================


## Взвешивание объектов. Sample weights (1 балл)

Иногда может оказаться, что не все объекты одинаково важны для обучения модели. Допустим у нас есть датасет с банковскими операциями, по которому мы должны научиться определять мошеннические операции. В этой выборке будут значительно преобладать законные операции, а мошеннических будет не так много (это кажется очевидным, если представить огромное количество операций в банке). Хочется сделать так, чтобы редко встречающиеся мошеннические операции помогали нам быстрее сдвигать градиент в нужную сторону, а значит и обучаться быстрее.

Чтобы воспользоваться этим, хочется увеличить удельный вес таких редких объектов в выборке. Это можно сделать двумя путями: либо чаще показывать такие объекты, либо добавить число, которое характеризует вес каждого такого объекта в общей выборке.

Здесь мы реализуем вторую идею с весам: добавим параметры `sample_weight` в методы `fit` и `partial_fit` (по умолчанию `None`), откуда будем передавать их в `sgd`. Внутри `sgd` же мы будем использовать вес каждого объекта, измненяя градиент на этом объекте пропорционально его весу.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestSampleWeights -v`

In [25]:
!pytest test_sgd.py -k TestSampleWeights -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 72 deselected / 1 selected                                

test_sgd.py::TestSampleWeights::test_no_weights SKIPPED (not impleme...) [100%]

====================== 1 skipped, 72 deselected in 0.57s =======================


## Коэффициент детерминации. $R^2$ (1 балл)

Выше мы уже реализовали метрики MAE и MSE, но они страдают одной проблемой -- они неинтерпретируемы. Поправим это дело.

У обертки реализуйте метод `score(X, y)`, который вычисляет коэффициент детерминации для своих аргументов. Вычисление должно происходить в два этапа: сначала внутри этого метода модель вычисляет предсказания для данного `X`, после чего вызывает функцию `r2_score`, которая вычисляет метрику на предсказанных значениях и таргетах.

Для начала реализуйте метрику `r2_score`, заготовку которой можно найти в файле `metrics.py`. После этого реализуйте описанную выше логику в методе `score` класса `SGDRegressor`. Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*): 

`pytest test_sgd.py -k TestR2Score -v`

In [26]:
!pytest test_sgd.py -k TestR2Score -v

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- /home/maksim/miniconda3/envs/mlngu/bin/python
cachedir: .pytest_cache
rootdir: /home/maksim/PycharmProjects/twinkle/03-sgd
plugins: anyio-3.5.0
collected 73 items / 71 deselected / 2 selected                                

test_sgd.py::TestR2Score::test_r2_score_set1 PASSED                      [ 50%]
test_sgd.py::TestR2Score::test_r2_score_set2 PASSED                      [100%]

======================= 2 passed, 71 deselected in 0.57s =======================


Подготовьте тренировочную и тестовые выборки, обучите на них модель, измерьте качество с помощью новой метрики.

In [27]:
from sklearn.datasets import load_boston
from StandardScaler import StandardScaler
from SGDRegressor import SGDRegressor
from metrics import r2_score
from sklearn.model_selection import train_test_split
from metrics import mean_squared_error, mean_absolute_error



X, y = load_boston(return_X_y=True)

In [28]:
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
reg = SGDRegressor(max_iter=1).fit(X_train, y_train)
prediction = reg.predict(X_test)
mse = mean_squared_error(prediction, y_test)
mae = mean_absolute_error(prediction, y_test)
print(f"MSE = {mse}\nMAE = {mae}\nR^2= {r2_score(y_true=y_test, y_pred=prediction)}")

MSE = 31.934652934745714
MAE = 4.018558449522879
R^2= 0.5780221674353194


* Какое значение получилось?
* Можно ли его как-то интерпретировать?

Модель неплохо объясняет данные

# Продолжение следует

Если вы успешно дошли до этой части, то вы проделали большую работу в плане понимания того, как работает алгоритм. Ковырять его мы продолжим в следующей домашней работе.